## 0. 데이터 로드, 전처리

<데이터 목록>
- 서울특별시_광진구_도시공원정보_20250310.csv
- 서울특별시 광진구_연령별인구현황_20241028.xlsx
- 세대원수별 세대수 통계.xlsx


### 0.1 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.font_manager as fm
import matplotlib as mpl

In [2]:
park_file = 'data/서울특별시_광진구_도시공원정보_20250310.csv'
park_data = pd.read_csv(park_file)
park_data.head(2)

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자
0,11215-10510,성화공원,어린이공원,NaN,서울특별시 광진구 자양동 199-4,37.536207,127.061859,253.0,NaN,조합놀이대,NaN,NaN,NaN,1979-09-15,서울특별시 광진구청,02-450-7775,2025-03-10
1,11215-10511,양마장공원,어린이공원,NaN,서울특별시 광진구 자양동 689-6,37.530871,127.088727,1113.0,NaN,"조합놀이대, 그네, 흔들놀이기구",NaN,NaN,NaN,1982-03-09,서울특별시 광진구청,02-450-7775,2025-03-10


In [3]:
age_file = 'data/서울특별시 광진구_연령별인구현황_20241028.xlsx'
age_data = pd.read_excel(age_file)
age_data.head(5)

,광진구,화양동,군자동,중곡1동,중곡2동,중곡3동,중곡4동,능동,구의1동,구의2동,구의3동,광장동,자양1동,자양2동,자양3동,자양4동,합계
0,0~4세,164,318,264,302,272,454,220,365,528,640,660,293,531,798,370,6179
1,5~9세,162,396,256,489,288,525,229,467,772,867,1716,398,641,1053,425,8684
2,10~14세,201,462,327,619,355,731,257,542,1017,1063,2752,613,801,1155,433,11328
3,15~19세,784,606,346,624,412,938,268,658,1164,1217,2551,806,842,1230,560,13006
4,20~24세,6565,2401,892,1111,779,1455,790,1646,1350,1589,1968,1481,1203,1590,1794,26614


In [4]:
households_file = 'data/세대원수별 세대수 통계.xlsx'
households_data = pd.read_excel(households_file)
households_data.head(5)

/Users/jisu/miniconda/envs/torch/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,동별(1),동별(2),2024,2024.1,2024.2,2024.3,2024.4,2024.5,2024.6,2024.7,2024.8,2024.9,2024.10
0,동별(1),동별(2),전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
1,합계,소계,4482063,2012137,1010838,746493,558273,119525,25828,6202,1799,572,396
2,종로구,소계,72166,38032,14704,10112,6944,1786,417,108,38,13,12
3,NaN,사직동,4557,2321,933,677,469,118,21,11,3,0,4
4,NaN,삼청동,1107,547,247,150,108,39,12,3,1,0,0


### 0.2 연령별인구 데이터  전처리

In [5]:
# import matplotlib.pyplot as plt

# # 한글 폰트 설정
# plt.rcParams['axes.unicode_minus'] = False

# # 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'

In [6]:
# import geopandas as gpd
# from shapely.geometry import Point
# import contextily as ctx
# import matplotlib.colors as colors

# # 1. 전국 행정동 경계 shp 파일 로드
# shp_file = 'data/통계청_SGIS 행정구역 통계 및 경계_20240630/2. 경계/3. 2024년 2분기 기준 행정동 경계/bnd_dong_00_2024_2Q.shp'
# korea_districts = gpd.read_file(shp_file)

# # 2. 광진구 행정동만 필터링 (ADM_CD가 111050으로 시작하는 행)
# # ADM_CD를 문자열로 변환하여 광진구 코드로 시작하는지 확인
# korea_districts['ADM_CD'] = korea_districts['ADM_CD'].astype(str)
# gwangjin_districts = korea_districts[korea_districts['ADM_CD'].str.startswith('11050')]

# print(f"광진구 행정동 수: {len(gwangjin_districts)}")
# print(gwangjin_districts[['ADM_NM', 'ADM_CD']])

# # 3. 공원 데이터를 GeoDataFrame으로 변환
# geometry = [Point(xy) for xy in zip(park_data['경도'], park_data['위도'])]
# park_gdf = gpd.GeoDataFrame(park_data, geometry=geometry, crs="EPSG:4326")

# # 4. 좌표계 통일
# if gwangjin_districts.crs != park_gdf.crs:
#     gwangjin_districts = gwangjin_districts.to_crs(park_gdf.crs)

# # 5. 시각화
# fig, ax = plt.subplots(figsize=(15, 15))

# # 행정동 경계 그리기 - 다른 색상으로 구분
# gwangjin_districts.plot(ax=ax, alpha=0.5, edgecolor='black', column='ADM_NM', legend=True, categorical=True)

# # 공원 구분별 색상 매핑
# color_dict = {'어린이공원': 'green', '근린공원': 'blue', '소공원': 'red', '기타': 'yellow'}
# cmap = colors.ListedColormap([color_dict[c] for c in park_gdf['공원구분'].unique()])

# park_gdf.plot(ax=ax,
#               column='공원구분', legend=True, 
#               categorical=True, marker='o',
#               alpha=0.7, edgecolor='black',
#               cmap=cmap)
# # 공원 이름 표시 
# for idx, row in park_gdf.iterrows():
#     ax.annotate(row['공원명'], 
#                 xy=(row.geometry.x, row.geometry.y),
#                 xytext=(3, 3),
#                 textcoords="offset points",
#                 fontsize=9)

# # 행정동 이름 표시
# for idx, row in gwangjin_districts.iterrows():
#     centroid = row.geometry.centroid
#     ax.annotate(row['ADM_NM'],
#                 xy=(centroid.x, centroid.y),
#                 ha='center', va='center',
#                 fontsize=12, fontweight='bold',
#                 bbox=dict(boxstyle="round,pad=0.3", fc='white', ec='gray', alpha=0.7))

# # 배경 지도 추가 (OSM)
# try:
#     ax.set_axis_off()
#     ctx.add_basemap(ax, crs=park_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
# except Exception as e:
#     print(f"배경 지도 추가 실패: {e}")

# plt.title('광진구 행정동별 공원 위치', fontsize=15)
# plt.tight_layout()
# plt.savefig('광진구_행정동별_공원위치.png', dpi=300)
# plt.show()

# # 6. 공간 조인을 통해 각 공원이 속한 행정동 찾기
# joined_parks = gpd.sjoin(park_gdf, gwangjin_districts, how="left", predicate="within")

In [7]:
# # 행정동 이름을 원본 데이터에 추가
# park_data['행정동'] = joined_parks['ADM_NM']
# park_data.head(2)

In [8]:
# age_data 전처리
age_data.fillna('')

age_data = age_data.rename(columns={'광진구': '행정동'})

age_data = age_data.T

# 이미 전치된 데이터프레임의 첫 번째 행을 칼럼명으로 설정
new_columns = age_data.iloc[0]

# 첫 번째 행을 제외한 데이터만 유지
age_data = age_data.iloc[1:]

# 마지막 행 삭제 (합계 행으로 보임)
age_data = age_data.iloc[:-1]

# 인덱스 재설정
age_data.columns = new_columns

# 인덱스를 재설정하여 일반 칼럼으로 만들기
age_data = age_data.reset_index()

# 이전 인덱스 칼럼명을 '행정동'으로 변경
age_data = age_data.rename(columns={'index': '행정동'})

# 칼럼 인덱스의 이름을 'index'로 변경
age_data.columns.name = 'index'

In [9]:
age_data

index,행정동,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,화양동,164,162,201,784,6565,7132,3650,1652,1164,...,980,861,796,598,491,327,184,48,11,2
1,군자동,318,396,462,606,2401,3277,2406,1474,1352,...,1333,1419,1176,783,653,485,224,59,11,4
2,중곡1동,264,256,327,346,892,1981,1946,1226,1081,...,1236,1180,1023,637,529,388,175,59,10,1
3,중곡2동,302,489,619,624,1111,2195,2210,1479,1483,...,1696,1690,1391,944,718,469,201,53,16,0
4,중곡3동,272,288,355,412,779,1540,1642,1159,1021,...,1352,1427,1173,729,644,434,194,45,13,0
5,중곡4동,454,525,731,938,1455,2380,2438,1796,1743,...,2685,2612,2205,1453,1082,741,331,103,24,6
6,능동,220,229,257,268,790,1548,1542,999,918,...,708,686,606,429,329,268,101,30,6,1
7,구의1동,365,467,542,658,1646,3265,2878,1824,1720,...,1754,1612,1304,816,681,481,247,71,18,5
8,구의2동,528,772,1017,1164,1350,1916,2073,1688,2047,...,2226,2073,1816,1134,949,615,324,95,14,7
9,구의3동,640,867,1063,1217,1589,2390,2445,2113,2088,...,2210,2066,1767,1134,825,582,336,117,31,3


In [10]:
numeric_columns = age_data.columns.tolist()

# '행정동'열만 제외
if '행정동' in numeric_columns:
    numeric_columns.remove('행정동')

# '전체 인구수' 열 추가 
age_data['전체 인구수'] = age_data[numeric_columns].sum(axis=1)

In [11]:
age_data

index,행정동,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,...,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상,전체 인구수
0,화양동,164,162,201,784,6565,7132,3650,1652,1164,...,861,796,598,491,327,184,48,11,2,27754
1,군자동,318,396,462,606,2401,3277,2406,1474,1352,...,1419,1176,783,653,485,224,59,11,4,21293
2,중곡1동,264,256,327,346,892,1981,1946,1226,1081,...,1180,1023,637,529,388,175,59,10,1,15587
3,중곡2동,302,489,619,624,1111,2195,2210,1479,1483,...,1690,1391,944,718,469,201,53,16,0,20603
4,중곡3동,272,288,355,412,779,1540,1642,1159,1021,...,1427,1173,729,644,434,194,45,13,0,15723
5,중곡4동,454,525,731,938,1455,2380,2438,1796,1743,...,2612,2205,1453,1082,741,331,103,24,6,27836
6,능동,220,229,257,268,790,1548,1542,999,918,...,686,606,429,329,268,101,30,6,1,11405
7,구의1동,365,467,542,658,1646,3265,2878,1824,1720,...,1612,1304,816,681,481,247,71,18,5,23686
8,구의2동,528,772,1017,1164,1350,1916,2073,1688,2047,...,2073,1816,1134,949,615,324,95,14,7,26042
9,구의3동,640,867,1063,1217,1589,2390,2445,2113,2088,...,2066,1767,1134,825,582,336,117,31,3,27992


In [12]:
# 새로운 데이터프레임 생성을 위한 코드

# 1. 모든 열 목록 가져오기
all_columns = age_data.columns.tolist()

# 2. '행정동' 열과 연령대 열 분리하기
age_columns = []
행정동_column = '행정동'

# 연령대 열만 필터링 (행정동 열 제외)
for col in all_columns:
    if col != '행정동' and col != '전체 인구수':
        age_columns.append(col)

# 3. 비율을 저장할 새로운 데이터프레임 생성
ratio_df = pd.DataFrame()

# 4. 행정동 열 복사
ratio_df['행정동'] = age_data['행정동']
ratio_df['전체 인구수'] = age_data['전체 인구수']

# 5. 각 연령대별 비율 계산
for col in age_columns:
    print(col)
    ratio_df[f'{col}_비율'] = age_data[col] / age_data['전체 인구수']

0~4세
5~9세
10~14세
15~19세
20~24세
25~29세
30~34세
35~39세
40~44세
45~49세
50~54세
55~59세
60~64세
65~69세
70~74세
75~79세
80~84세
85~89세
90~94세
95~99세
100세 이상


In [13]:
ratio_df

,행정동,전체 인구수,0~4세_비율,5~9세_비율,10~14세_비율,15~19세_비율,20~24세_비율,25~29세_비율,30~34세_비율,35~39세_비율,...,55~59세_비율,60~64세_비율,65~69세_비율,70~74세_비율,75~79세_비율,80~84세_비율,85~89세_비율,90~94세_비율,95~99세_비율,100세 이상_비율
0,화양동,27754,0.005909,0.005837,0.007242,0.028248,0.236542,0.256972,0.131513,0.059523,...,0.03531,0.031023,0.028681,0.021546,0.017691,0.011782,0.00663,0.001729,0.000396,0.000072
1,군자동,21293,0.014934,0.018598,0.021697,0.02846,0.11276,0.1539,0.112995,0.069225,...,0.062603,0.066642,0.055229,0.036773,0.030667,0.022777,0.01052,0.002771,0.000517,0.000188
2,중곡1동,15587,0.016937,0.016424,0.020979,0.022198,0.057227,0.127093,0.124848,0.078655,...,0.079297,0.075704,0.065632,0.040867,0.033939,0.024893,0.011227,0.003785,0.000642,0.000064
3,중곡2동,20603,0.014658,0.023734,0.030044,0.030287,0.053924,0.106538,0.107266,0.071786,...,0.082318,0.082027,0.067514,0.045819,0.034849,0.022764,0.009756,0.002572,0.000777,0.0
4,중곡3동,15723,0.017299,0.018317,0.022578,0.026204,0.049545,0.097946,0.104433,0.073714,...,0.085989,0.090759,0.074604,0.046365,0.040959,0.027603,0.012339,0.002862,0.000827,0.0
5,중곡4동,27836,0.01631,0.01886,0.026261,0.033697,0.05227,0.085501,0.087584,0.064521,...,0.096458,0.093835,0.079214,0.052199,0.038871,0.02662,0.011891,0.0037,0.000862,0.000216
6,능동,11405,0.01929,0.020079,0.022534,0.023498,0.069268,0.13573,0.135204,0.087593,...,0.062078,0.060149,0.053135,0.037615,0.028847,0.023498,0.008856,0.00263,0.000526,0.000088
7,구의1동,23686,0.01541,0.019716,0.022883,0.02778,0.069493,0.137845,0.121506,0.077008,...,0.074052,0.068057,0.055054,0.034451,0.028751,0.020307,0.010428,0.002998,0.00076,0.000211
8,구의2동,26042,0.020275,0.029644,0.039052,0.044697,0.051839,0.073573,0.079602,0.064818,...,0.085477,0.079602,0.069734,0.043545,0.036441,0.023616,0.012441,0.003648,0.000538,0.000269
9,구의3동,27992,0.022864,0.030973,0.037975,0.043477,0.056766,0.085382,0.087346,0.075486,...,0.078951,0.073807,0.063125,0.040512,0.029473,0.020792,0.012003,0.00418,0.001107,0.000107


### 0.3 행정동별 세대원수 전체 통계 구하기

In [14]:
# 열 이름 변경하기
households_data = households_data.rename(columns={
    '동별(1)': '행정구',
    '동별(2)': '행정동',
    '2024': '전체세대',
    '2024.1': '1인세대',
    '2024.2': '2인세대',
    '2024.3': '3인세대',
    '2024.4': '4인세대',
    '2024.5': '5인세대',
    '2024.6': '6인세대',
    '2024.7': '7인세대',
    '2024.8': '8인세대',
    '2024.9': '9인세대',
    '2024.10': '10인세대 이상'
})
households_data

,행정구,행정동,전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
0,동별(1),동별(2),전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
1,합계,소계,4482063,2012137,1010838,746493,558273,119525,25828,6202,1799,572,396
2,종로구,소계,72166,38032,14704,10112,6944,1786,417,108,38,13,12
3,NaN,사직동,4557,2321,933,677,469,118,21,11,3,0,4
4,NaN,삼청동,1107,547,247,150,108,39,12,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,NaN,천호2동,18355,9484,4345,2541,1570,310,70,23,9,3,0
449,NaN,길동,23493,10844,5545,3803,2552,565,129,43,9,1,2
450,NaN,강일동,13810,3771,3695,2946,2544,721,113,15,3,2,0
451,NaN,상일1동,13565,2064,3031,3822,3830,668,115,23,5,3,4


In [15]:
households_data = households_data.iloc[2:]
households_data

,행정구,행정동,전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
2,종로구,소계,72166,38032,14704,10112,6944,1786,417,108,38,13,12
3,NaN,사직동,4557,2321,933,677,469,118,21,11,3,0,4
4,NaN,삼청동,1107,547,247,150,108,39,12,3,1,0,0
5,NaN,부암동,4135,1675,1025,732,500,139,47,13,3,0,1
6,NaN,평창동,7134,2268,1837,1446,1114,355,79,22,10,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,NaN,천호2동,18355,9484,4345,2541,1570,310,70,23,9,3,0
449,NaN,길동,23493,10844,5545,3803,2552,565,129,43,9,1,2
450,NaN,강일동,13810,3771,3695,2946,2544,721,113,15,3,2,0
451,NaN,상일1동,13565,2064,3031,3822,3830,668,115,23,5,3,4


In [16]:
# 행정구 열의 NaN 값을 앞의 유효한 값으로 채우기
households_data['행정구'] = households_data['행정구'].fillna(method='ffill')

# 변경된 데이터프레임 확인
households_data.head(20)

/var/folders/2w/lkcqygdx72s_vktqq1wmb9p40000gn/T/ipykernel_77759/1013685481.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  households_data['행정구'] = households_data['행정구'].fillna(method='ffill')
/var/folders/2w/lkcqygdx72s_vktqq1wmb9p40000gn/T/ipykernel_77759/1013685481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  households_data['행정구'] = households_data['행정구'].fillna(method='ffill')


,행정구,행정동,전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
2,종로구,소계,72166,38032,14704,10112,6944,1786,417,108,38,13,12
3,종로구,사직동,4557,2321,933,677,469,118,21,11,3,0,4
4,종로구,삼청동,1107,547,247,150,108,39,12,3,1,0,0
5,종로구,부암동,4135,1675,1025,732,500,139,47,13,3,0,1
6,종로구,평창동,7134,2268,1837,1446,1114,355,79,22,10,2,1
7,종로구,무악동,2986,704,763,702,636,146,27,4,2,1,1
8,종로구,교남동,4302,1603,1060,897,604,119,11,5,1,2,0
9,종로구,가회동,1875,898,410,302,201,43,17,2,1,0,1
10,종로구,종로1.2.3.4가동,4898,3911,498,273,161,39,11,4,0,1,0
11,종로구,종로5.6가동,3533,2533,501,293,161,37,5,3,0,0,0


In [17]:
kg_households_data = households_data[households_data['행정구'] == '광진구']
kg_households_data

,행정구,행정동,전체세대,1인세대,2인세대,3인세대,4인세대,5인세대,6인세대,7인세대,8인세대,9인세대,10인세대 이상
71,광진구,소계,169931,85851,35640,25311,18309,3685,838,213,60,12,12
72,광진구,화양동,18011,14938,1906,674,397,66,24,5,0,0,1
73,광진구,군자동,11223,7056,2037,1207,724,140,39,14,4,2,0
74,광진구,중곡1동,8673,4957,1943,1030,578,130,29,6,0,0,0
75,광진구,중곡2동,10548,5310,2479,1509,963,222,51,9,2,2,1
76,광진구,중곡3동,8473,4530,2029,1087,647,148,23,8,1,0,0
77,광진구,중곡4동,13001,5414,3390,2280,1484,322,82,23,4,1,1
78,광진구,능동,6452,3957,1279,665,414,93,36,8,0,0,0
79,광진구,구의1동,13191,7812,2589,1537,980,220,39,11,1,1,1
80,광진구,구의2동,11471,4474,2832,2125,1578,353,81,19,5,0,4


### 0.4 행정동별 전체 인구수를 행정동별 전체 세대수로 나눠서 ratio_df에 추가하는 코드를 작성

In [18]:
# 행정동별 전체 인구수를 행정동별 전체 세대수로 나눠서 ratio_df에 추가하는 코드를 작성

# 행정동별 인구/세대 비율 계산

# 1. 두 데이터프레임을 행정동을 기준으로 병합
# ratio_df에는 인구수가 있고, kg_households_data에는 세대수가 있음
merged_df = pd.merge(ratio_df, kg_households_data[['행정동', '전체세대']], on='행정동', how='left')

# 2. 행정동별 인구/세대 비율 계산하여 새 열 추가
merged_df['인구/세대_비율'] = merged_df['전체 인구수'] / merged_df['전체세대']

# 3. 필요한 열만 선택하여 ratio_df 업데이트
ratio_df = merged_df

# 결과 확인
ratio_df

,행정동,전체 인구수,0~4세_비율,5~9세_비율,10~14세_비율,15~19세_비율,20~24세_비율,25~29세_비율,30~34세_비율,35~39세_비율,...,65~69세_비율,70~74세_비율,75~79세_비율,80~84세_비율,85~89세_비율,90~94세_비율,95~99세_비율,100세 이상_비율,전체세대,인구/세대_비율
0,화양동,27754,0.005909,0.005837,0.007242,0.028248,0.236542,0.256972,0.131513,0.059523,...,0.028681,0.021546,0.017691,0.011782,0.00663,0.001729,0.000396,0.000072,18011,1.540947
1,군자동,21293,0.014934,0.018598,0.021697,0.02846,0.11276,0.1539,0.112995,0.069225,...,0.055229,0.036773,0.030667,0.022777,0.01052,0.002771,0.000517,0.000188,11223,1.897265
2,중곡1동,15587,0.016937,0.016424,0.020979,0.022198,0.057227,0.127093,0.124848,0.078655,...,0.065632,0.040867,0.033939,0.024893,0.011227,0.003785,0.000642,0.000064,8673,1.797187
3,중곡2동,20603,0.014658,0.023734,0.030044,0.030287,0.053924,0.106538,0.107266,0.071786,...,0.067514,0.045819,0.034849,0.022764,0.009756,0.002572,0.000777,0.0,10548,1.953261
4,중곡3동,15723,0.017299,0.018317,0.022578,0.026204,0.049545,0.097946,0.104433,0.073714,...,0.074604,0.046365,0.040959,0.027603,0.012339,0.002862,0.000827,0.0,8473,1.855659
5,중곡4동,27836,0.01631,0.01886,0.026261,0.033697,0.05227,0.085501,0.087584,0.064521,...,0.079214,0.052199,0.038871,0.02662,0.011891,0.0037,0.000862,0.000216,13001,2.141066
6,능동,11405,0.01929,0.020079,0.022534,0.023498,0.069268,0.13573,0.135204,0.087593,...,0.053135,0.037615,0.028847,0.023498,0.008856,0.00263,0.000526,0.000088,6452,1.767669
7,구의1동,23686,0.01541,0.019716,0.022883,0.02778,0.069493,0.137845,0.121506,0.077008,...,0.055054,0.034451,0.028751,0.020307,0.010428,0.002998,0.00076,0.000211,13191,1.795618
8,구의2동,26042,0.020275,0.029644,0.039052,0.044697,0.051839,0.073573,0.079602,0.064818,...,0.069734,0.043545,0.036441,0.023616,0.012441,0.003648,0.000538,0.000269,11471,2.270247
9,구의3동,27992,0.022864,0.030973,0.037975,0.043477,0.056766,0.085382,0.087346,0.075486,...,0.063125,0.040512,0.029473,0.020792,0.012003,0.00418,0.001107,0.000107,12040,2.324917


In [19]:
ratio_columns = ratio_df.columns.tolist()

remove_columns = ['행정동', '전체 인구수', '전체세대', '인구/세대_비율']

for col in remove_columns:
    ratio_columns.remove(col)


for col in ratio_columns:
    # 비율 열에서 "_비율"을 제거하여 새 열 이름 생성
    new_col = col.replace('_비율', '')
    
    # 인구/세대_비율과 곱하여 새 열 생성
    ratio_df[new_col] = ratio_df[col] * ratio_df['인구/세대_비율']

ratio_df = ratio_df.drop(columns=ratio_columns)
ratio_df

,행정동,전체 인구수,전체세대,인구/세대_비율,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,화양동,27754,18011,1.540947,0.009106,0.008995,0.01116,0.043529,0.364499,0.39598,...,0.054411,0.047804,0.044195,0.033202,0.027261,0.018156,0.010216,0.002665,0.000611,0.000111
1,군자동,21293,11223,1.897265,0.028335,0.035285,0.041165,0.053996,0.213936,0.29199,...,0.118774,0.126437,0.104785,0.069767,0.058184,0.043215,0.019959,0.005257,0.00098,0.000356
2,중곡1동,15587,8673,1.797187,0.030439,0.029517,0.037703,0.039894,0.102848,0.22841,...,0.142511,0.136054,0.117952,0.073446,0.060994,0.044737,0.020178,0.006803,0.001153,0.000115
3,중곡2동,20603,10548,1.953261,0.028631,0.046359,0.058684,0.059158,0.105328,0.208096,...,0.160789,0.16022,0.131873,0.089496,0.06807,0.044463,0.019056,0.005025,0.001517,0.0
4,중곡3동,15723,8473,1.855659,0.032102,0.03399,0.041898,0.048625,0.091939,0.181754,...,0.159566,0.168417,0.13844,0.086038,0.076006,0.051222,0.022896,0.005311,0.001534,0.0
5,중곡4동,27836,13001,2.141066,0.03492,0.040382,0.056226,0.072148,0.111914,0.183063,...,0.206523,0.200908,0.169602,0.111761,0.083224,0.056996,0.02546,0.007922,0.001846,0.000462
6,능동,11405,6452,1.767669,0.034098,0.035493,0.039833,0.041538,0.122443,0.239926,...,0.109733,0.106324,0.093924,0.066491,0.050992,0.041538,0.015654,0.00465,0.00093,0.000155
7,구의1동,23686,13191,1.795618,0.02767,0.035403,0.041089,0.049882,0.124782,0.247517,...,0.132969,0.122205,0.098855,0.06186,0.051626,0.036464,0.018725,0.005382,0.001365,0.000379
8,구의2동,26042,11471,2.270247,0.046029,0.0673,0.088658,0.101473,0.117688,0.16703,...,0.194055,0.180717,0.158312,0.098858,0.08273,0.053613,0.028245,0.008282,0.00122,0.00061
9,구의3동,27992,12040,2.324917,0.053156,0.07201,0.088289,0.10108,0.131977,0.198505,...,0.183555,0.171595,0.146761,0.094186,0.068522,0.048339,0.027907,0.009718,0.002575,0.000249


In [20]:
# ratio_df 데이터프레임을 CSV 파일로 저장
ratio_df.to_csv('행정동별_인구비율_데이터.csv', index=False, encoding='utf-8-sig')

print('데이터프레임이 "행정동별_인구비율_데이터.csv" 파일로 저장되었습니다.')

데이터프레임이 "행정동별_인구비율_데이터.csv" 파일로 저장되었습니다.
